# Tutorial on model selection/sparsification 
using 1D polynomial regression as an example.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model

# Generate Data
We shall do a 1D polynomial regression problem: assume $y = f(x,\mathbf{p}) = \Theta(x) \cdot \mathbf{p}$, where $\Theta(x)$ is a library of polynomials of $x$, we want to infer $p$ from some observations of ${\hat{x},\hat{y}}$ pairs. Assume $\hat{y}$ is contaminated by some Gaussian noise of known variance $\sigma$.

First, let's generate some data.

1. Select some sampling points in $x$
2. Create an arbitrary polynomial function with sparse terms
3. Add noise to $y$

In [ ]:
# Generate sample data
x = np.linspace(-1, 1, 100)
y = #(fill in) # Some random polynomial function of x
sigma = 0.5 # Standard deviation of noise - Play with it!
yData = y + sigma * np.random.normal(size=x.shape)

# Wider sample point (for checking extrapolation)
x_wide = #(fill in)

Let's visualise what we have generated

In [ ]:
# Plot the data
plt.scatter(x, yData, label='Data Points', color='blue', s=10)
plt.plot(x, y, label='True Function', color='black')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Sample Data with Noise')
plt.legend()
plt.show()

# Create a polynomial library
To prepare for the regression, let's create some helper function to evaluate the polynomial.

In [ ]:
def polynomial_library(x, degree):
    """
    Generate a library of polynomial features up to the given degree for input x.
    Returns a matrix where each column is x**i for i in [0, degree].
    """
    return #(fill in) # Fill in to return the polynomial feature matrix [ 1 x x**2 x**3 ... x**degree ]

In [ ]:
# Create polynomial features up to degree 7 - output: N x (degree+1) matrix
Theta = polynomial_library(x,20)

# Ordinary Least Square regression
The easiest way to do regression is, of course, by linear least square.

Let's try that

In [ ]:
# Regression to get coefficients p (OLS)
p_OLS = #(fill in)
# Predicted output using OLS coefficients
y_OLS = Theta @ p_OLS

print("OLS Coefficients:", p_OLS) # Are the coefficients close to the true polynomial coefficients?

Let's visualise.

In [ ]:
# Plot the result
plt.scatter(x, yData, label='Data Points', color='blue', s=10)
plt.plot(x, y, label='True Function', color='black')
plt.plot(x, y_OLS, label='OLS', color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

### Food for thoughts

1. Does it fit well? 
2. What if we increase $\sigma$? 
3. What if we decrease the number of data points?
4. What if we increase the degree in the polynomial library?
   - How should we determine the library size?
5. What if we extrapolate to outside the sampled range?

What is the key to a good model?

# Regularised Regression (Ridge,LASSO)
Now, let's try to redo the above with some regularisation.


In [ ]:
alpha = #(fill in) # Regularization strength for Ridge regression
# Regression to get coefficients p (Ridge)
p_Ridge = #(fill in)
# Predicted output using Ridge coefficients
y_Ridge = Theta @ p_Ridge

print("Ridge Coefficients:", p_Ridge) # Are the coefficients close to the true polynomial coefficients?


In [ ]:
alpha_LASSO = #(fill in) # Regularization strength for LASSO regression
# Regression to get coefficients p (Lasso)
p_Lasso = linear_model.Lasso(alpha=(alpha_LASSO * sigma**2), fit_intercept=False).fit(Theta, yData).coef_
# Predicted output using Lasso coefficients
y_Lasso = Theta @ p_Lasso

print("Lasso Coefficients:", p_Lasso) # Are the coefficients close to the true polynomial coefficients?

### Food for thoughts

1. Which regularisation give more zero coefficients (sparsification)?
2. Which one gives better prediction?
   - Which one generalise better outside the sampled domain?
3. Which one is more noise robust (try change the noise)?
4. How should we determine the value of $\alpha$ in both regularisation?

# Maximising Evidence by Greedy search
In this section, we will employ the idea of maximising evidence to select the right term in the library. The idea is to start from the full library, and remove terms one by one according to which removal maximise the evidence.

To create this algorithm, we will need to
1. Implement a function that calculate the evidence given the active terms
2. Implement a Greedy search loop

We will assume ridge regression in this algorithm (but feel free to try other regulariser too!).

In [ ]:
# Function to compute the regression and the evidence
def evidence(Theta,target, alpha, sigma2):
    """
    Compute the log-evidence of the model given design matrix Theta, target data,
    regularization parameter alpha, and noise standard deviation sigma.
    """
    A = #(fill in) # Matrix to be inverted to compute ridge regression
    coef = np.linalg.solve(A, (Theta.T @ target)/sigma2)
    
    JMap = #(fill in) # Compute the negative log-posterior at the MAP estimate
    JEvidence = JMap + #(fill in) Log(det(A/2/pi)) # Compute the negative log-evidence
    return coef, JEvidence


In [ ]:
mask = np.ones(Theta.shape[1], dtype=bool)
coef = np.zeros((Theta.shape[1],Theta.shape[1]))
evidence_list = np.zeros(Theta.shape[1])
alpha = #(fill in) # Regularization strength for Evidence regression

# Greedy Search - Search for the best combination of polynomial terms that maximizes the evidence
for i in range(Theta.shape[1]):
    #(fill in)
    for j in range(Theta.shape[1]):
        #(fill in)

    #(fill in)

# Final selected model
#(fill in)
print("\n Best model coeff:", p_Evidence)

In [ ]:
# Plot the result
plt.scatter(x, yData, label='Data Points', color='blue', s=10)
plt.plot(x, y, label='True Function', color='black')
plt.plot(x, y_Evidence, label='Evidence', color='green')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

### Food for thoughts

1. How would you determine $\alpha$?
2. What happens when we play with $\sigma$?
3. What would you do if you do not know $\sigma$ a priori?

# (optional) SparseBayes / ARD algorithm
SparseBayes (a.k.a. ARDRegression) is an automatic algorithm that optimise the prior variance and sparsify the model by maiximising the evidence.

In [ ]:
# Regression to get coefficients p (Ridge)
p_ARD = linear_model.ARDRegression().fit(Theta, yData).coef_
# Predicted output using Ridge coefficients
y_ARD = Theta @ p_ARD

print("ARD Coefficients:", p_ARD) # Are the coefficients close to the true polynomial coefficients?

In [ ]:
# Plot the result
plt.scatter(x, yData, label='Data Points', color='blue', s=10)
plt.plot(x, y, label='True Function', color='black')
plt.plot(x, y_ARD, label='ARD', color='red')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

### Food for thoughts
How does it compare with your implementation of the Bayesian Evidence Maximisation?